In [29]:
import pandas as pd
from IPython.display import display, HTML
import numpy as np
from sklearn.utils import resample

# Create a connection with SQL server to get data.
def exec_sql_query(query, param=None):
    
    from sqlalchemy import create_engine
    import urllib
    params = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};SERVER=LAPTOP-C3LFVOFI;DATABASE=student_life;UID=student_sense;PWD=abhinav123")
    engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
    connection = engine.raw_connection()
    
    try:
        cursor = connection.cursor()
        if(param):
            cursor.execute(query, param)
        else : 
            cursor.execute(query)
            
        results = cursor.fetchall()
        columns = [column[0] for column in cursor.description]
        df = pd.DataFrame.from_records(results, columns=columns)
        cursor.close()
        connection.commit()
    finally:
        connection.close()
    
    del engine
    return df

In [30]:
from IPython.display import display
import numpy as np

# Getting Raw Data.

activity_details_raw = exec_sql_query("select student_id, activity_time, activity_inference from  activity_details where student_id = 1")
stress_details_raw = exec_sql_query("select student_id, response_time, stress_level from stress_details where student_id = 1")



In [31]:
# SLicing.
activity_details =  activity_details_raw.loc[:,["activity_time","activity_inference", "student_id"]]
stress_details = stress_details_raw.loc[:,["response_time","stress_level","student_id"]] 

activity_details.rename({"activity_time":"time"}, axis="columns", inplace=True)
stress_details.rename({"response_time":"time"}, axis="columns", inplace=True)

# Sorting.
activity_details = activity_details.sort_values(by="time")
stress_details = stress_details.sort_values(by="time")

activity_details = activity_details.dropna()
# stress_details.dropna(inplace=True)

display(activity_details.head())
display(stress_details.head())

,time,activity_inference,student_id
92494,2013-03-27 04:01:39,0,1
92495,2013-03-27 04:01:41,0,1
92496,2013-03-27 04:01:44,0,1
92497,2013-03-27 04:01:46,0,1
92498,2013-03-27 04:01:49,0,1


,time,stress_level,student_id
1,2013-03-24 09:47:00,1,1
2,2013-03-24 10:18:43,3,1
3,2013-03-24 10:30:03,4,1
4,2013-03-24 10:48:22,3,1
0,2013-03-27 04:45:45,4,1


In [32]:
activity_train_x = pd.merge_asof(stress_details,
                                 activity_details,
                                 direction="nearest",
                                 by="student_id",
                                 on="time",
                                 tolerance=pd.Timedelta("1 hour"))

display(activity_train_x)

,time,stress_level,student_id,activity_inference
0,2013-03-24 09:47:00,1,1,NaN
1,2013-03-24 10:18:43,3,1,NaN
2,2013-03-24 10:30:03,4,1,NaN
3,2013-03-24 10:48:22,3,1,NaN
4,2013-03-27 04:45:45,4,1,0.0
5,2013-03-28 23:53:09,4,1,0.0
6,2013-03-29 19:45:25,1,1,0.0
7,2013-03-30 00:32:58,5,1,0.0
8,2013-03-30 22:27:26,1,1,0.0
9,2013-03-31 18:49:27,1,1,0.0


In [33]:
# Creating New column for activity_details
activity_details["stress_level"] = np.nan

# Appending other rows.
activity_train_x = activity_train_x.loc[:,["time", "student_id", "activity_inference", "stress_level"]]

activity_train_x = activity_train_x.append(activity_details.loc[:,["time", "student_id", "activity_inference", "stress_level"]],
                        ignore_index=True)

activity_train_x.sort_values(by=["student_id","time"], inplace=True)
activity_train_x.drop_duplicates()
activity_train_x.activity_inference.fillna(method="backfill", inplace=True)


In [34]:
# Seeing Final Data.

not_null = activity_train_x.loc[:,"stress_level"].notnull()
print(activity_train_x.head())

# Distributing the data into train X and train y.
activity_train_x_final = activity_train_x.loc[:,"activity_inference"]
activity_train_y_final = activity_train_x.loc[:,"stress_level"]

activity_train_x_final.to_csv("data/activity_train_x")
activity_train_y_final.to_csv("data/activity_train_y")

                  time  student_id  activity_inference  stress_level
0  2013-03-24 09:47:00           1                 0.0           1.0
1  2013-03-24 10:18:43           1                 0.0           3.0
2  2013-03-24 10:30:03           1                 0.0           4.0
3  2013-03-24 10:48:22           1                 0.0           3.0
36 2013-03-27 04:01:39           1                 0.0           NaN
